In [ ]:
# Clone your GitHub repository and install required packages
!git clone https://github.com/arman-rn/MLDL-SemSeg.git
%cd MLDL-SemSeg

# Install necessary Python packages
!pip install -U fvcore gdown==4.7.1 --quiet

Cloning into 'RTDA-SemanticSegmentation'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 141 (delta 75), reused 107 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 920.30 KiB | 7.02 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/RTDA-SemanticSegmentation
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import zipfile

# Unzip Cityscapes dataset
with zipfile.ZipFile('/content/drive/MyDrive/MLDL_PROJECT/Cityscapes.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

# Unzip GTA5 dataset
with zipfile.ZipFile('/content/drive/MyDrive/MLDL_PROJECT/GTA5.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

# Verify that the files are extracted
!ls /content/data/Cityscapes
!ls /content/data/GTA5

Cityspaces
images	labels


In [3]:
import os

# Path to the root of your downloaded GTA5 dataset (must match Cell 3 & config.py)
gta5_data_root_for_preprocess = "/content/data/GTA5" # Or your actual path

preprocess_script_path = "datasets/preprocess_gta5_labels.py" # Assuming it's at the root of your repo

if not os.path.exists(preprocess_script_path):
    print(f"⚠️ ERROR: '{preprocess_script_path}' not found in your cloned repository at '{os.getcwd()}'.")
else:

    # Prepend current directory to PYTHONPATH for the script execution
    !PYTHONPATH=. python {preprocess_script_path} \
        --gta5_root {gta5_data_root_for_preprocess}
    print("✅ GTA5 label pre-conversion script finished.")


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Color-to-ID LUT for GTA5 initialized for pre-processing.
Saving pre-converted ID labels to: /content/data/GTA5/labels_trainids
Converting labels: 100% 2500/2500 [02:38<00:00, 15.80it/s]

Pre-conversion complete. 2500 labels processed.
Converted labels saved in: /content/data/GTA5/labels_trainids
✅ GTA5 label pre-conversion script finished.


In [ ]:
import wandb
wandb.login()

In [5]:
import sys
import importlib

# --- Re-import config to pick up manual changes ---
try:
    import config as cfg
    importlib.reload(cfg) # Make sure the manually edited paths are loaded

except ImportError:
    print("ERROR: Could not import config.py. Ensure it's in your project path and sys.path is correct.")
    raise
except Exception as e:
    print(f"ERROR during config reload or path check: {e}")
    raise

# --- Run main.py's logic ---
# Store original sys.argv to restore later if needed
original_argv = sys.argv.copy()


sys.argv = [
    'main.py'
]
print(f"\nRunning main script with simulated command line arguments: {sys.argv}")

try:
    import main as project_main
    importlib.reload(project_main) # Re-import main in case its imports (like config) changed
    project_main.main()
    print("\n--- Full training run has finished (or attempted to finish). ---")
except Exception as e:
    import traceback
    print(f"An error occurred during the execution of main(): {e}")
    print("Full traceback:")
    print(traceback.format_exc())
finally:
    # Restore original sys.argv
    sys.argv = original_argv

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()



Running main script with simulated command line arguments: ['main.py']
Seeds set to 42
--- Effective Configuration (from cfg object after CLI overrides) ---
Model Name: BISENET
Train Dataset: GTA5 (1280x720)
  - Path: /content/data/GTA5
Validation Dataset: CITYSCAPES (1024x512)
  - Path: /content/data/Cityscapes/Cityspaces
GTA5 Transformations: ['Resize', 'ColorJitter', 'Normalize', 'ToTensorV2']
BiSeNet Context Path: resnet18
Optimizer: ADAM
Base Learning Rate (for adam): 0.0001
Weight Decay: 0.0001
Training for 50 epochs.
Device: cuda
Batch Size (from config): 8
Checkpoint Directory: ./checkpoints/bisenet
Resume from checkpoint: No
-----------------------------


Weights & Biases initialized successfully.
Loading GTA5 training data from: /content/data/GTA5
Using GTA5 training transforms (resize to 1280x720).
GTA5 will use pre-converted labels from subdir: 'labels_trainids'
Found 2500 image-label pairs in /content/data/GTA5
Found 2500 training images for gta5.
Loading Cityscapes validation data from: /content/data/Cityscapes/Cityspaces
Using Cityscapes validation transforms (resize to 1024x512).
Found 500 validation images for cityscapes.
Train loader: 312 batches, 2500 images.
Val loader: 500 batches, 500 images.
Loading BiSeNet model with 19 classes.
Using BiSeNet context path: resnet18


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 176MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 212MB/s]
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


Model 'bisenet' moved to device: cuda
Using mixed-precision training with GradScaler on CUDA device.
No resume checkpoint specified. Starting training from scratch.

--- Epoch 1/50 ---


...


Validation Epoch 50: Avg Loss: 0.8456, Overall Mean IoU: 0.2979

--- Training Finished ---
Loading best model from ./checkpoints/bisenet/best_miou_checkpoint.pth for final evaluation...
Loading checkpoint from './checkpoints/bisenet/best_miou_checkpoint.pth'...
Checkpoint loaded successfully.
Loaded per-class IoUs from the best checkpoint.

Calculating performance metrics (FLOPs, Latency on current model state)...


context_path.features.avgpool, context_path.features.fc, supervision1, supervision2


Calculating FLOPs and Parameters...
Calculating Latency (warmup: 10 iters, measurement: 100 iters)...
 Latency iteration 10/100 completed.
 Latency iteration 20/100 completed.
 Latency iteration 30/100 completed.
 Latency iteration 40/100 completed.
 Latency iteration 50/100 completed.
 Latency iteration 60/100 completed.
 Latency iteration 70/100 completed.
 Latency iteration 80/100 completed.
 Latency iteration 90/100 completed.
 Latency iteration 100/100 completed.

--- Final Results for Run: Step3b_ColorJitter ---
| Best Overall mIoU on CITYSCAPES (%) | 31.62 |
| Latency (ms) @ 1024x512 | 4.74 +/- 0.17 |
| FLOPs (G) @ 1024x512    | 25.78 |
| Parameters (M)               | 12.58 |

Per-Class IoUs from Best Model Checkpoint:
| Class Name           | IoU     |
|----------------------|---------|
| road                 | 0.8563 |
| sidewalk             | 0.2824 |
| building             | 0.7378 |
| wall                 | 0.2307 |
| fence                | 0.1408 |
| pole                 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
train/batch_loss,█▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch_loss,█▅▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,█████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▁▁▁
val/epoch_loss,█▄▃▃▅▃▂▄▃▄▂▂▁▂▂▁▃▂▃▁▁▂▁▁▁▁▁▁▂▁▁▁▁▂▁▂▂▁▂▂
val/mIoU,▁▄▃▅▃▆▆▅▆▅▇▇▆▆▇▆▅▇█▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇
val_iou_per_class/iou_bicycle,▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▂▃▃▃▁▄█▆▂▅▄▃▅▆▄▆▅▅▆▅
val_iou_per_class/iou_building,▁▅▅▃▅▇▄▂▅▇▇▄▇▇▆▃▆██▆█▆█▆█▆▇▇▇▇▆▆▆▆▆▆▆▆▇▆
val_iou_per_class/iou_bus,▂▁▅▂█▂▆▃▂▇▇▁▂▂▃▄▃▃▃▂▆▃▄▅▂▄▃▄▃▄▃▃▃▃▄▃▄▃▄▄
val_iou_per_class/iou_car,▂▂▄▂▁▆▃▅▄▅▃▅▃▅▄▄▆▅▇█▇▅▆▆▆▆▆▇█▇▇▇▆▇▇▇▇▇▇▇
val_iou_per_class/iou_fence,▆▇▁▆▂▄█▆▇▁▅▃▇▄▂▇▂▆▄▄▅▄▆▄▆▆▄▃▄▄▃▄▄▄▅▄▄▄▄▄


Run completed.

--- Full training run has finished (or attempted to finish). ---
